In [1]:
#initial data loading and cleaning

import pandas as pd
from pandas.io.json import json_normalize
import requests
import json

country = pd.read_csv("country.csv")
country.columns = ["country_code","country_name"]

time = pd.read_csv("timezone.csv")
time.columns = ["zone_id","abbreviation","time_start","gmt_offset","dst"]

zone = pd.read_csv("zone.csv")
zone.columns = ["zone_id","country_code","zone_name"]

langCode = pd.read_csv("langCode.csv")
langCode = langCode.iloc[:,2:4]

url = 'countryLang.json'

df = pd.read_json(url)
df = df.transpose()
df = df.reset_index()
top2lang = pd.DataFrame(df['languages'].apply(pd.Series)).iloc[:,0:2]
countryLang = pd.concat([pd.DataFrame(df["name"]),top2lang],axis = 1)
countryLang.columns = ["name","primLang","secLang"]
print(langCode.head(5))
print(countryLang.head(5))


  Alpha2_Code English_Name
0          aa         Afar
1          ab    Abkhazian
2         NaN     Achinese
3         NaN        Acoli
4         NaN      Adangme
                   name primLang secLang
0               Andorra       ca     NaN
1  United Arab Emirates       ar     NaN
2           Afghanistan       ps      uz
3   Antigua and Barbuda       en     NaN
4              Anguilla       en     NaN


In [2]:
#initial joins, want language name on language code

#do not want missing Alpha2_codes on initial join
countryLangCode = pd.merge(langCode.dropna(),countryLang,left_on='Alpha2_Code',right_on = 'primLang',how = 'inner')
countryLangCode = pd.merge(countryLangCode,langCode,left_on = "secLang",right_on = 'Alpha2_Code',how = 'inner')

countryLangCode = countryLangCode.iloc[:,[2,1,3,6,4]]
countryLangCode.columns = ["country","codePrimary","namePrimary","codeSecondary","nameSecondary"]



In [3]:
#time Zone joins

timeZone = pd.merge(zone,time,on = 'zone_id',how = 'inner')

timeZone = timeZone.sort_values(by = ['time_start'],ascending = True)
print(timeZone.shape)
#rid of time_start
offset = pd.read_csv("timezonesOffset.csv")

timeZone = pd.merge(timeZone,offset,left_on= 'zone_name',right_on = 'timezone',how = 'inner')
print(timeZone.columns)

timeZone = timeZone.iloc[:,[1,2,7,9]]
timeZone['offsetHours'] = timeZone.iloc[:,3]/3600

countryTimeZone = pd.merge(timeZone,country,on = "country_code",how = 'inner')
countryTimeZone["country_name"]=countryTimeZone["country_name"].str.replace(pat = "Russian Federation",repl="Russia")


(161976, 7)
Index(['zone_id', 'country_code', 'zone_name', 'abbreviation', 'time_start',
       'gmt_offset', 'dst', 'timezone', 'offset', 'offset_dst'],
      dtype='object')


In [4]:
countryTimeZoneLang =  pd.merge(countryTimeZone,countryLangCode,left_on = 'country_name',right_on = 'country',how = 'inner')
countryTimeZoneLang = countryTimeZoneLang.iloc[:,[5,0,1,2,3,4,7,8,9,10]]

In [5]:
zone_split = countryTimeZoneLang["zone_name"].str.split("/",n=1,expand = True)

In [6]:
countryTimeZoneLang["zone_name"] = zone_split[1]

In [7]:
countryTimeZoneLang.sort_values("country_name",ascending = False,inplace = True)



In [8]:
countryTimeZoneLang.rank(method = 'first')



ValueError: first not supported for non-numeric data